In [ ]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen

from tqdm import tqdm_notebook
import unidecode

In [ ]:
df_debut = pd.read_csv("../data/debut_album_1118-utf8.csv")
df_debut.head()

In [ ]:
billboard_df = pd.DataFrame(columns = ['artist', 'album', 'billboard_frequency'])

for i in tqdm_notebook(range(len(df_debut['artist']))):
    
    artist = df_debut['artist'][i]
    album = df_debut['album'][i]

    artist_ = artist.replace(" ", "%20")
    album_ = album.replace(" ", "%20")

    keyword = artist_ + "%20" + album_
    
    url = "https://www.billboard.com/search/" + keyword

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    try:
        article_count = int(str(soup.select("#main > div > div > div.section.article-section > div.section-header > div.section-header__count")[0].get_text())[10:-1])
    except IndexError:
        article_count = 0
    
    data = {
    'artist' : artist,
    'album' : album,
    'billboard_frequency' : article_count,
    }
    
    billboard_df.loc[len(billboard_df)] = data

In [ ]:
print(billboard_df.shape)
billboard_df.head()

In [ ]:
billboard_df['artist'] = billboard_df['artist'].apply(lambda x: unidecode.unidecode(x))
billboard_df['album'] = billboard_df['album'].apply(lambda x: unidecode.unidecode(x))
billboard_df['artist'] = billboard_df['artist'].apply(lambda x : x.replace(",", ""))
billboard_df['album'] = billboard_df['album'].apply(lambda x : x.replace(",", ""))

In [ ]:
billboard_df.loc[2, 'album'] = '9am to 5pm 5pm to Whenever'
billboard_df.loc[529, 'album'] = 'If You Love Me'
billboard_df.loc[1143, 'album'] = 'Gover Tela Body Language'
billboard_df.loc[1411, 'album'] = 'Fresh Filter Volumn 1'

In [ ]:
billboard_df.shape

In [ ]:
billboard_df.to_csv("../data/buzz/buzz_billboard.csv", index=False, encoding="UTF-8")

In [ ]:
df = pd.read_csv("../data/buzz/buzz_billboard.csv")
df.shape

### add to mysql database

In [ ]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [ ]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [ ]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [ ]:
df.to_sql(name="buzz_billboard", con=engine, if_exists='replace')